In [1]:
!pip install langchain
!pip install datasets
!pip install faiss-cpu
%pip install -qU langchain-google-genai
!pip install -U jq
%pip install -qU langchain-huggingface
!pip install langchain-community langchain-core
!pip install tiktoken
!unzip /content/codebert_finetuned.zip -d .
!unzip /content/processed_repositories.zip -d .

Выходные данные были обрезаны до нескольких последних строк (5000).
   creating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/ADDRESSES_PROVIDER.txt  
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/getReserveConfigurationData.txt  
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/getAllReservesTokens.txt  
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/getUserReserveData.txt  
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/getAllATokens.txt  
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/getReserveTokensAddresses.txt  
  inflating: ./processed_repositories/2021-12-sublime/src/IProtocolDataProvider/getReserveData.txt  
   creating: ./processed_repositories/2021-12-sublime/src/IVerification/
  inflating: ./processed_repositor

In [2]:
from transformers import RobertaTokenizer, RobertaModel
import torch

class SolidityEmbeddings:
    def __init__(self):
        self.tokenizer = RobertaTokenizer.from_pretrained('.')
        self.model = RobertaModel.from_pretrained('.')
        self.model.eval()

    def __call__(self, text):
        tokens = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**tokens)
        # Mean pool across the sequence dimension for fixed-size embedding
        return outputs.last_hidden_state.mean(dim=1).squeeze(0)

solidityembeddings = SolidityEmbeddings()

Some weights of RobertaModel were not initialized from the model checkpoint at . and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# document loaders
import os
import json

functions_code = []
repos_code_path='/content/processed_repositories'
for root, _, files in os.walk(repos_code_path):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
                try:
                  code=json_data["Code"]
                  functions_code.append(code)
                except:
                  print(file, "hasnt got Code")
                  continue

print(functions_code[11])

nction slice(
        bytes memory _bytes,
        uint256 _start,
        uint256 _length
    )
        internal
        pure
        returns (bytes memory)
    {
        require(_length + 31 >= _length, "slice_overflow");
        require(_bytes.length >= _start + _length, "slice_outOfBounds");

        bytes memory tempBytes;

        assembly {
            switch iszero(_length)
            case 0 {
                // Get a location of some free memory and store it in tempBytes as
                // Solidity does for memory variables.
                tempBytes := mload(0x40)

                // The first word of the slice result is potentially a partial
                // word read from the original array. To read it, we calculate
                // the length of that partial word and start copying that many
                // bytes into the array. The first word we copy will start with
                // data we don't care about, but the last `lengthmod` bytes will
                

In [4]:
print(len(functions_code))

21299


In [5]:
query = ["contract MathOperations { \
         // Pure function to add two numbers \
         function addNumbers(uint a, uint b) public pure returns (uint) { \
         return a + b; \
          } \
        }"]

docs_embeddings = solidityembeddings(functions_code[:5])
query_embedding = solidityembeddings(query).unsqueeze(0)

print(docs_embeddings.shape)
print(query_embedding.shape)

torch.Size([5, 768])
torch.Size([1, 768])


In [6]:
doc_vector_norm = docs_embeddings / docs_embeddings.norm(dim=1, keepdim=True)
query_vector_norm = query_embedding / query_embedding.norm(dim=1, keepdim=True)

cosine_similarity = torch.matmul(doc_vector_norm, query_vector_norm.T)  # torch.Size([B, 1])

print("Cosine Similarity:", cosine_similarity)

Cosine Similarity: tensor([[0.4864],
        [0.9710],
        [0.4867],
        [0.9719],
        [0.4809]])


In [ ]:
# vector store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

documents=[]
for i in range(len(functions_code[:50])):
  if isinstance(functions_code[i],str):explain planetoid cora dataset
    documents.append(Document(page_content=functions_code[i],metadata={"source":"codearena"}))
print(len(documents))

100


In [11]:
dim = solidityembeddings(query).shape[-1]  # Embedding dimensionality
index = faiss.IndexFlatL2(dim)

vector_store = FAISS(
    embedding_function=solidityembeddings,
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={},
)

vector_store.add_documents(documents=documents)

KeyboardInterrupt: 

In [ ]:
# retriever

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.95},
)
retriever.invoke(query)

[Document(metadata={'source': 'codearena'}, page_content='function getReserves()\n        external\n        view\n        returns (\n            uint112 reserve0,\n            uint112 reserve1,\n            uint32 blockTimestampLast\n        );'),
 Document(metadata={'source': 'codearena'}, page_content='function burn(address to)\n        external\n        returns (uint256 amount0, uint256 amount1);'),
 Document(metadata={'source': 'codearena'}, page_content='function swap(\n        uint256 amount0Out,\n        uint256 amount1Out,\n        address to,\n        bytes calldata data\n    ) external;')]